<a href="https://colab.research.google.com/github/cxbxmxcx/TensorFlow_Certification/blob/master/TensorFlow_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q tensorflow-text

     |████████████████████████████████| 3.0MB 2.5MB/s 


In [1]:
import tensorflow_datasets as tfds
import tensorflow as tf


In [21]:
dataset, info = tfds.load('imdb_reviews/subwords8k', with_info=True,
                          as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

In [3]:
encoder = info.features['text'].encoder
encoder.vocab_size

8185

In [4]:
encoded = encoder.encode("Hello how are you today.")
encoded

[4025, 222, 113, 29, 37, 1076, 7975]

In [5]:
[' ---> '.join([str(index),encoder.decode([index])]) for index in encoded]

['4025 ---> Hell',
 '222 ---> o ',
 '113 ---> how ',
 '29 ---> are ',
 '37 ---> you ',
 '1076 ---> today',
 '7975 ---> .']

In [22]:
BUFFER_SIZE = 100000
BATCH_SIZE = 64


train_dataset = train_dataset.shuffle(BUFFER_SIZE)

train_dataset = train_dataset.take(5000)
test_dataset = test_dataset.take(1000)

train_dataset = train_dataset.padded_batch(BATCH_SIZE)
test_dataset = test_dataset.padded_batch(BATCH_SIZE)


In [10]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(encoder.vocab_size, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),    
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1)
])

In [16]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [23]:
history = model.fit(train_dataset, epochs=10)

Epoch 1/10
79/79 [==============================] - 14s 175ms/step - loss: 0.5542 - accuracy: 0.6780
Epoch 2/10
79/79 [==============================] - 14s 182ms/step - loss: 0.4668 - accuracy: 0.8087
Epoch 3/10
79/79 [==============================] - 14s 179ms/step - loss: 0.4352 - accuracy: 0.8317
Epoch 4/10
79/79 [==============================] - 14s 181ms/step - loss: 0.3640 - accuracy: 0.8725
Epoch 5/10
79/79 [==============================] - 14s 171ms/step - loss: 0.5067 - accuracy: 0.7388
Epoch 6/10
79/79 [==============================] - 14s 181ms/step - loss: 0.6096 - accuracy: 0.5821
Epoch 7/10
79/79 [==============================] - 14s 178ms/step - loss: 0.6005 - accuracy: 0.5906
Epoch 8/10
79/79 [==============================] - 14s 179ms/step - loss: 0.6034 - accuracy: 0.5827
Epoch 9/10
79/79 [==============================] - 14s 181ms/step - loss: 0.5948 - accuracy: 0.5952
Epoch 10/10
79/79 [==============================] - 14s 182ms/step - loss: 0.5940 - accura

In [33]:
def sample_predict(sample_pred_text):
  encoded_sample_pred_text = encoder.encode(sample_pred_text)  
  encoded_sample_pred_text = tf.cast(encoded_sample_pred_text, tf.float32)
  predictions = model.predict(tf.expand_dims(encoded_sample_pred_text, 0))

  return (predictions)

In [34]:
sample_pred_text = ('The movie was cool. The animation and the graphics '
                    'were out of this world. I would recommend this movie.')
predictions = sample_predict(sample_pred_text)
print(sum(predictions[0]))

[4.2505746]


In [35]:
# predict on a sample text with padding

sample_pred_text = ('I hated this movie. The animation and the graphics '
                    'were terrible. I would not recommend this movie.')
predictions = sample_predict(sample_pred_text)
print(sum(predictions[0]))

[-11.319085]
